In [7]:
#API Kagle
#Instalando o pacote
!pip install kagglehub

import os
from pathlib import Path

#Corrigindo o JSON que extraimos do Kaggle
kaggle_creds = """{
    "username": "matheussilveirasilva",
    "key": "d45f520dcd5bd26a2351e73145f53a54"
}"""

#Escrevendo as credenciais
(kaggle_dir / "kaggle.json").write_text(kaggle_creds)

#Configurando permissões
!chmod 600 {kaggle_dir / "kaggle.json"}

#Realizando o Download
import kagglehub
path = kagglehub.dataset_download("mhassansaboor/toyota-motors-stock-data-2980-2024")
print("Path to dataset files:", path)

In [4]:
#Análise exploratoria para verificarmos se precisamos fazer alguma alteração no arquivo:

import pandas as pd

#Leitura do arquivo
toyota = pd.read_csv(r'C:\Users\mathe\.cache\kagglehub\datasets\mhassansaboor\toyota-motors-stock-data-2980-2024\versions\1\Toyota_Data.csv')
nulos_por_coluna = toyota.isnull().sum()

#Analise exploratoria do arquivo

#Primeiras linhas do arquivo
print ('Primeiras linhas do arquivo:')
print (toyota.head())
print ('')
print ('')
print ('')
#Detalhes do arquivo como quantidade de linhas, nulls e Dtype:
print ('Detalhes do arquivo:')
print (toyota.info())
print ('')
print ('')
print ('')
#Detalhes estatisticos do arquivo, como média, mediana e moda:
print ('Estatisticas descritivas do arquivo:')
print (toyota.describe())
print ('')
print ('')
print ('')
#Valores nulos, já verificamos no Head, mas vale a pena ter certeza:
print ('Busca por valores nulos entre as tabelas:')
if nulos_por_coluna.sum() == 0:
    print ('Não há valores nulos')
else:
    print ('Valores nulos encontrados')
    for coluna, quantidade in nulos_por_coluna.itens():
        if quantidade > 0:
            print (f'Coluna {coluna}: quantidade {quantidade} nulos')

Primeiras linhas do arquivo:
         Date  Adj Close     Close      High       Low      Open  Volume
0  1980-03-17   2.296798  3.291227  3.344743  3.291227  0.000000   41109
1  1980-03-18   2.306134  3.304606  3.358122  3.304606  0.000000    9343
2  1980-03-19   2.306134  3.304606  3.304606  3.304606  3.304606       0
3  1980-03-20   2.306134  3.304606  3.358122  3.304606  0.000000   10277
4  1980-03-21   2.362154  3.384880  3.438396  3.384880  0.000000    8409



Detalhes do arquivo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11291 entries, 0 to 11290
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       11291 non-null  object 
 1   Adj Close  11291 non-null  float64
 2   Close      11291 non-null  float64
 3   High       11291 non-null  float64
 4   Low        11291 non-null  float64
 5   Open       11291 non-null  float64
 6   Volume     11291 non-null  int64  
dtypes: float64(5), int64(1), object(1)
m

In [31]:
#Alterando tipo dos documentos
import pandas as pd

#Leitura do arquivo
toyota = pd.read_csv(r'C:\Users\mathe\.cache\kagglehub\datasets\mhassansaboor\toyota-motors-stock-data-2980-2024\versions\1\Toyota_Data.csv')

#Alterações para melhor entendimento da tabela
toyota.columns = ['Data', 'Fechamento Ajustado', 'Fechamento', 'Alta', 'Baixa', 'Abertura', 'Volume']
#Garantindo que a coluna data esteja em datetime e convertendo para o padrão mundial
toyota ['Data'] = pd.to_datetime(toyota['Data'])
toyota['Data'] = toyota['Data'].dt.strftime('%d-%m-%Y')

#Salvar o arquivo novo
output_file = r'C:\Users\mathe\OneDrive\Documents\portfolio\toyota\toyota_data.csv'
#Salvar o DataFrame combinado em um novo arquivo CSV
toyota.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')


print(f"Arquivos combinados com sucesso! O arquivo resultante foi salvo em: {output_file}")


Arquivos combinados com sucesso! O arquivo resultante foi salvo em: C:\Users\mathe\OneDrive\Documents\portfolio\toyota\toyota_data.csv


In [43]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# .ENV
load_dotenv("C:\\Users\\mathe\\bd_toyota.env")

# URL
DATABASE_URL = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

# Criar engine de conexão
engine = create_engine(DATABASE_URL)

# teste
try:
    with engine.connect() as connection:
        print("Ok!")
except Exception as e:
    print(f"Erro: {e}")

Erro: 'utf-8' codec can't decode byte 0xe7 in position 78: invalid continuation byte


In [21]:
from sqlalchemy import create_engine
# from sqlalchemy.pool import NullPool
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv('bd_toyota.env')

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# Construct the SQLAlchemy connection string
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URL)
# If using Transaction Pooler or Session Pooler, we want to ensure we disable SQLAlchemy client side pooling -
# https://docs.sqlalchemy.org/en/20/core/pooling.html#switching-pool-implementations
# engine = create_engine(DATABASE_URL, poolclass=NullPool)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!


In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# 1. Configuração da Conexão ===============================================
load_dotenv('bd_toyota.env')

USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"
engine = create_engine(DATABASE_URL)

# 2. Carregar CSV e Preparar Dados =========================================
try:
    df = pd.read_csv('toyota_data.csv', delimiter=';', encoding='utf-8')

    # Converter nomes das colunas para snake_case (evita problemas no PostgreSQL)
    df.columns = (
        df.columns
        .str.lower()              # Tudo para minúsculas
        .str.replace(' ', '_')    # Substituir espaços por underscore
        .str.replace(r'[^\w]', '')  # Remover caracteres especiais
    )

    # Converter coluna de data
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)

    # Converter Volume para inteiro
    df['volume'] = df['volume'].astype(int)

    print("\n✅ Dados carregados com sucesso!")

except Exception as e:
    print(f"\n❌ Erro ao carregar CSV: {e}")
    exit()

# 3. Enviar Dados para o Banco =============================================
try:
    df.to_sql(
        name='toyota_stock',
        con=engine,
        if_exists='append',
        index=False,
        method='multi',
        chunksize=500
    )

    print("\n✅ Dados enviados para o banco!")

except Exception as e:
    print(f"\n❌ Erro ao enviar dados: {e}")



✅ Dados carregados com sucesso!

✅ Dados enviados para o banco!
